In [1]:
import pandas as pd
import sqlite3 as sql

In [30]:
mydb = 'Abstracts_aug4.db'

In [23]:
jsonFile = "json42.json"

# Helper Functions to inspect database tables and json file

In [ ]:
def jsonDF(jsonFile):
    f = open(jsonFile, "r+")
    return pd.read_json(f, orient='index')
    

In [4]:
def getContents():
    '''
    : param NONE
    : output : Returns a json dictionary of the table names, entry counts, and links to tables 
                of all table names in the database
    ''' 
    with sql.connect(mydb) as con:
    
        cursor = con.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        mytables = (cursor.fetchall())
        myt = [t[0] for t in mytables]
        
        return myt
            

In [5]:
def sqlCMDToPD(table, 
               db):
    '''Take a sql db and return as a readable pandas DataFrame
       : param db : str. Name of db. (ie. 'Abstracts.db'
       : param sqlcmd : str. Sqlite3 cmd to execute. 
               default: "SELECT * FROM Abstracts" 
                           > select all from Abstracts table
    '''
    
    #connect to a db
    with sql.connect(db) as con:
        
        #run command
        sqlcmd = "SELECT * FROM '%s'" %table
        df = pd.read_sql_query(sqlcmd, con)
        
        # Check resulting pandas DF shape
        print df.shape
        
        return df


In [ ]:
t = getContents()
t

In [31]:
c = sqlCMDToPD("CONFERENCES", mydb)
t = sqlCMDToPD("ABSTRACTSTOTAL", mydb)
pap = sqlCMDToPD("PAPER", mydb)
pub = sqlCMDToPD("PUBLICATIONS", mydb)
k = sqlCMDToPD("KEYS", mydb)

au = sqlCMDToPD("AUTHORS", mydb)
aff = sqlCMDToPD("AFFILIATIONS", mydb)

pk = sqlCMDToPD("PAPERKEY", mydb)
ap = sqlCMDToPD("AFFILIATIONPAPER", mydb)
pa = sqlCMDToPD("PAPERAUTHOR", mydb)
t.tail()


(3, 2)
(1140, 7)
(1102, 8)
(27, 3)
(5557, 2)
(1949, 2)
(830, 2)
(17436, 2)
(1102, 2)
(3432, 2)


,Abstract,Author affiliation,Authors,Conf,Title,terms,year
1135,Reference architectures have emerged as a spec...,"Dept of Computer Systems, University of S o...","['Guessi, Milena', '', '', 'Oquendo, Flavio', ...",WICSA,Variability viewpoint to describe reference ar...,"['Software architecture', 'Architectural viewp...",2014
1136,The architecture of a software system plays a ...,"Software Architecture Research Group, Universi...","['Stevanetic, Srdjan', '', '', 'Javed, Muhamma...",WICSA,Empirical evaluation of the understandability ...,"['Computer software', 'Graphic methods', 'Soft...",2014
1137,Global Software Development GSD teams encoun...,"Software and Systems Group, IT University of C...","['Chauhan, Muhammad Aufeef']",WICSA,A reference architecture for providing tools a...,"['Cloud computing', 'Distributed computer syst...",2014
1138,Software architectures and their representatio...,"ABB Corporate Research Germany, Industrial Sof...","['Sehestedt, Stephan', '', '', 'Cheng, Chih Ho...",WICSA,Towards quantitative metrics for architecture ...,"['Software architecture', 'Sustainable develop...",2014
1139,Up to date architecture views help to better u...,"RWTH Aachen University, Research Group Softwar...","['Dragomir, Ana', '', '', 'Harun, M', 'Firdaus...",WICSA,On bridging the gap between practice and visio...,"['Computer software', 'Architecture descriptio...",2014


#  Functions using sqlite3 and python to insert and delete and update tables.


In [38]:
def insert_toTable(db, df, table = 'ABSTRACTSTOTAL'):
    '''Check to insert a new record into a database table, inserts if does not exist
    param  db str : Database name to connect to
    param df pandas dataframe : dataframe being inspected for entry
    param table str : Table Name to insert into, if does not exist will create
    output : new entry inserted
    '''
    with sql.connect(db) as con:
        df.to_sql(table, con, flavor='sqlite', 
                      schema=None, if_exists='append',
                      index=False, index_label=None,
                      chunksize=None, dtype=None)
            
        print("Records %s inserted"%table)
    
def insertcheckRecord(db, df, table = 'CONFERENCES', un = 'confName' ):
    '''Check to insert a new record into a database table, inserts if does not exist
    param  db str : Database name to connect to
    param df pandas dataframe : dataframe being inspected for entry
    param table str : Table Name to insert into, if does not exist will create
    param un str : unique column to check for entry to create a new pk, if not will just append
    '''
    t = sqlCMDToPD(table, mydb)
    if df[un][0] not in t[un].unique():
        insert_toTable(db, df[un], table)
    else:
        print(" %s already exists, try upserting with key value or deleting" %df[un][0])

    
def insertcheckRecordTWO(db, df, table = 'PUBLICATIONS', un = 'confName', un1 = 'year' ):
    '''Check to insert a new record into a database table, inserts if does not exist, checks for multiple
    entries as unique
    param  db str : Database name to connect to
    param df pandas dataframe : dataframe being inspected for entry
    param table str : Table Name to insert into, if does not exist will create
    param un str : unique column to check for entry to create a new pk, if not will just append
    param un1 str : unique column2 to check for entry to create a new pk, if fail un, if not will just append
    '''
    t = sqlCMDToPD(table, mydb)
    if df[un][0] not in t[un].unique():
        print df[un][0], 'is new'
        insert_toTable(db, df[[un, un1]], table)
    else:
        conf = df[un][0]
        tmp = t.query('@conf == confName') 
        
        if df[un1][0] not in tmp[un1].unique():
            print (df[[un, un1]].values), 'is a new entry'
            insert_toTable(db, df[[un, un1]], table)
        else:
            print(" %s already exists, try upserting with key value or deleting" %df[[un, un1]].values)


def insertValues(db, table, value1, value2):
    '''Insert a new record by value into a database table
    param  db str : Database name to connect to
    param table str : Table Name to insert into, if does not exist will create
    param value str : unique value entered into table
    '''
    with sql.connect(db) as con:
        try:
            con.execute("INSERT INTO {tn} VALUES('%s','%s')".format(tn=table)%(value1, value2))
            print('%s %s inserted into %s')%(value1, value2, table)
        except:
            con.execute("INSERT INTO {tn} VALUES(Null,'%s')".format(tn=table)%(value2))
            print('%s inserted into %s, single entry')%(value2, table)

        
def enterValueCheck_nested(db, table, values, cn):
    '''Check to insert a new record into a database table, inserts if does not exist
    param  db str : Database name to connect to
    param table str : Table Name to insert into, if does not exist will create
    param values python series : series being parsed and formated to inspection and entry into table
    param cn str : column name to check for entry to create a new pk
    '''
    keys = []
    tableDF = sqlCMDToPD(table, db)
    for i, ky in enumerate(values):
        for key in ky.split(','):
            if key not in tableDF[cn].unique():
                print key, 'is new'
                insertValues(db, table, None, key)
            else:
                print key, 'already exists in table'
            
            keys.append(key)
        return keys

def compositeCreation(db, table1, col, values, parentID, comptable):
    '''Creating Composite Tables 
    First, find the values needed to insert from first table based on query
    then insert each (parentID, value) pair into the composite table 
    param  db str : Database name to connect to
    param table1 str : Table Name to query for multiple values
    param col str : column name of value to retrieve iteratively 
    param values list : list of values to insert into the composite
    param parentID int : integer value (Primary Key Value) of parent table to enter
    param comptable str : Table Name of composite table
    '''
    t = sqlCMDToPD(table1, db)
    tmp = t.query('{cn} in @values'.format(cn = col))[col]
    for v in tmp.values:
        print v, paperK
        insertValues(db, comptable, parentID, v)
        
def getPK(db, table, pkCol):
    '''retrieve the PRIMARY KEY
    param  db str : Database name to connect to
    param table str : Table Name to delete from
    param pkcol str : primary column name being used, 
    '''
    with sql.connect(db) as c:
        cursor = c.cursor()
        cursor.execute("SELECT {idf} FROM {tn} ORDER BY {idf} DESC LIMIT 1".format(tn=table, idf=pkCol))
        key = cursor.fetchone()[0]

        return key
    
def deleteRowPK(db, table, pkcol, entryID):
    '''Deleting a Record by PRIMARY KEY
    param  db str : Database name to connect to
    param table str : Table Name to delete from
    param pkcol str : primary column name being used, 
    param entryID int : integer value (Primary Key Value) to delete from table
    '''
    with sql.connect(db) as con:
        
        con.execute("DELETE FROM {tn} WHERE {idf}={my_id}".format(tn=table, idf=pkcol, my_id=entryID))

        con.commit()

def deleteRowOTHER(db, table, cn, entry):
    '''Deleting a Record
    param  db str : Database name to connect to
    param table str : Table Name to delete from
    param cn str : column name being used for deletion comparason (if no PK column, ie, abstracts Total)
    param entry str : the str to be used to find and remove records (removes all records
    '''
    with sql.connect(db) as con:
        
        con.execute("DELETE FROM {tn} WHERE {idf}='%s'".format(tn=table, idf=cn)%entry)

        con.commit()
        
def entryintotables(db, jsonfile):
    '''Inserting a Record from a JsonFile
    param  db str : Database name to connect to
    param jsonfile str : name of Json File to be read into the database
    '''
    f = open(jsonfile, "r+")
    jdf = pd.read_json(f, orient='index')
    #TOTALABSTRACTS, check and then insert if needed, uniqueness based on Abstract column
    insert_toTable(db, jdf, table = 'ABSTRACTSTOTAL')
    


    #renaming of columns
    jdf.rename(columns = {'Conf':'confName'}, inplace= True)
    jdf.rename(columns = {'Author affiliation' : 'affiliation'}, inplace = True)
    jdf.rename(columns = {'Authors' : 'authors'}, inplace = True)
    
    #CONFERENCES, check and then insert if needed, uniqueness based on Abstract column
    insertcheckRecord(db, jdf, table = 'CONFERENCES', un = 'confName' )
    
    #PUBLICATIONS, check and then insert if needed, uniqueness based on Abstract column
    insertcheckRecordTWO(db, jdf, table = 'PUBLICATIONS', un = 'confName', un1 = 'year' )
    
    #AFFILIATIONS
    insertcheckRecord(db = db, df = jdf, table = 'AFFILIATIONS', un = 'affiliation')
    
    #For the nested: authors, keywords, and need to reparse/reformat, also to show numerous ways to insert:
    #KEYS
    keys = enterValueCheck_nested(db=db, table = 'KEYS', values = jdf.terms, cn = 'keyword')
    
    #AUTHORS
    authors = enterValueCheck_nested(db=db, table = 'AUTHORS', values = jdf.authors, cn = 'authorName')
    
    #PAPER
    jdf.rename(columns = {'year' : 'pubYear'}, inplace = True)
    insert_toTable(mydb, jdf, 'PAPER')
    paperID  = getPK(db, 'PAPER', 'paperID') 
    
    #COMPOSITE TABLE UPDATES
    #PAPERKEY
    compositeCreation(db, 'KEYS', 'keyword', keys, paperID, 'PAPERKEY')
    
    #PAPERAUTHOR
    compositeCreation(db, 'AUTHORS', 'authorName', authors, paperID, 'PAPERAUTHOR')
    
    #AFFILIATIONPAPER
    affilationID = getPK(db, 'AFFILIATIONS', 'affilID' )
    insertValues(mydb, "AFFILIATIONPAPER", paperID, affilationID)
    
    
    return jdf, keys, authors

In [39]:
j, keylist, authorslist = entryintotables(mydb, jsonFile )

Records ABSTRACTSTOTAL inserted
(3, 2)
 ECSA already exists, try upserting with key value or deleting
(27, 3)
 [[u'ECSA' 2009]] already exists, try upserting with key value or deleting
(830, 2)
Records AFFILIATIONS inserted
(5557, 2)
one potato is new
one potato inserted into KEYS, single entry
 two potato is new
 two potato inserted into KEYS, single entry
 three potato is new
 three potato inserted into KEYS, single entry
 four is new
 four inserted into KEYS, single entry
(1949, 2)
mr brown is new
mr brown inserted into AUTHORS, single entry
 mr blue is new
 mr blue inserted into AUTHORS, single entry
 mr mogatu is new
 mr mogatu inserted into AUTHORS, single entry
Records PAPER inserted
(5561, 2)
one potato 1103
1103 one potato inserted into PAPERKEY
 two potato 1103
1103  two potato inserted into PAPERKEY
 three potato 1103
1103  three potato inserted into PAPERKEY
 four 1103
1103  four inserted into PAPERKEY
(1952, 2)
mr brown 1103
1103 mr brown inserted into PAPERAUTHOR
 mr blue

In [40]:
c = sqlCMDToPD("CONFERENCES", mydb)
t = sqlCMDToPD("ABSTRACTSTOTAL", mydb)
pap = sqlCMDToPD("PAPER", mydb)
pub = sqlCMDToPD("PUBLICATIONS", mydb)
k = sqlCMDToPD("KEYS", mydb)

au = sqlCMDToPD("AUTHORS", mydb)
aff = sqlCMDToPD("AFFILIATIONS", mydb)

pk = sqlCMDToPD("PAPERKEY", mydb)
ap = sqlCMDToPD("AFFILIATIONPAPER", mydb)
pa = sqlCMDToPD("PAPERAUTHOR", mydb)
pa.tail()
t.tail()

(3, 2)
(1143, 7)
(1103, 8)
(27, 3)
(5561, 2)
(1952, 2)
(831, 2)
(17440, 2)
(1103, 2)
(3435, 2)


,Abstract,Author affiliation,Authors,Conf,Title,terms,year
1138,Software architectures and their representatio...,"ABB Corporate Research Germany, Industrial Sof...","['Sehestedt, Stephan', '', '', 'Cheng, Chih Ho...",WICSA,Towards quantitative metrics for architecture ...,"['Software architecture', 'Sustainable develop...",2014
1139,Up to date architecture views help to better u...,"RWTH Aachen University, Research Group Softwar...","['Dragomir, Ana', '', '', 'Harun, M', 'Firdaus...",WICSA,On bridging the gap between practice and visio...,"['Computer software', 'Architecture descriptio...",2014
1140,"For real, i mean Rice Krispie Treat Cereal and...",Some Fancy Pants Nancy Place or My cereal bowl,"mr brown, mr blue, mr mogatu",ECSA,Monty Python and the Holy Grail,"one potato, two potato, three potato, four",2009
1141,"For real, i mean Rice Krispie Treat Cereal and...",Some Fancy Pants Nancy Place or My cereal bowl,"mr brown, mr blue, mr mogatu",ECSA,Monty Python and the Holy Grail,"one potato, two potato, three potato, four",2009
1142,"For real, i mean Rice Krispie Treat Cereal and...",Some Fancy Pants Nancy Place or My cereal bowl,"mr brown, mr blue, mr mogatu",ECSA,Monty Python and the Holy Grail,"one potato, two potato, three potato, four",2009


In [25]:
#t.tail()
jtest = pd.read_json(jsonFile, orient='index')
jtest

,Abstract,Author affiliation,Authors,Conf,Title,terms,year
abstract,"For real, i mean Rice Krispie Treat Cereal and...",Some Fancy Pants Nancy Place or My cereal bowl,"mr brown, mr blue, mr mogatu",ECSA,Monty Python and the Holy Grail,"one potato, two potato, three potato, four",2009


In [ ]:

deleteRowOTHER(mydb, "PAPER", 'title', 'Monty Python and the Holy Grail' )
pap2 = sqlCMDToPD("PAPER", mydb)
pap2.tail()

In [ ]:
keys = sqlCMDToPD('KEYS', mydb)
keys.tail()

In [ ]:
deleteRowPK(mydb, 'KEYS', 'keyID', 5562)
deleteRowPK(mydb, 'KEYS', 'keyID', 5561)
deleteRowPK(mydb, 'KEYS', 'keyID', 5560)
deleteRowPK(mydb, 'KEYS', 'keyID', 5558)
deleteRowPK(mydb, 'AUTHORS', 'authorID', 1950)
deleteRowPK(mydb, 'AUTHORS', 'authorID', 1951)
deleteRowPK(mydb, 'AUTHORS', 'authorID', 1952)
deleteRowPK(mydb, 'AUTHORS', 'authorID', 1950)
deleteRowPK(mydb, 'AFFILIATIONS', 'affilID', 831)
keys = sqlCMDToPD('KEYS', mydb)
au = sqlCMDToPD('AUTHORS', mydb)
aff = sqlCMDToPD('AFFILIATIONS', mydb)
aff.tail()

# Looking at some other sqlite functions

In [41]:
def retrievals(db, table_name, column_2, column_3, id_column, some_id ):
    with sql.connect(db) as con:
        c =  cursor = con.cursor()
        # 1) Contents of all columns for row that match a certain value in 1 column
        c.execute('SELECT * FROM {tn} WHERE {cn}="ECSA"'.\
                  format(tn=table_name, cn=column_2))
        all_rows = c.fetchall()
        print('1):', all_rows)

    # 2) Value of a particular column for rows that match a certain value in column_1
        c.execute('SELECT ({coi}) FROM {tn} WHERE {cn}="ECSA"'.\
                  format(coi=column_2, tn=table_name, cn=column_2))
        all_rows = c.fetchall()
        print('2):', all_rows)

    # 3) Value of 2 particular columns for rows that match a certain value in 1 column
        c.execute('SELECT {coi1},{coi2} FROM {tn} WHERE {coi1}="ECSA"'.\
                  format(coi1=column_2, coi2=column_3, tn=table_name, cn=column_2))
        all_rows = c.fetchall()
        print('3):', all_rows)

    # 4) Selecting only up to 10 rows that match a certain value in 1 column
        c.execute('SELECT * FROM {tn} WHERE {cn}="ECSA" LIMIT 10'.\
                  format(tn=table_name, cn=column_2))
        ten_rows = c.fetchall()
        print('4):', ten_rows)

    # 5) Check if a certain ID exists and print its column contents
        c.execute("SELECT * FROM {tn} WHERE {idf}={my_id}".\
                  format(tn=table_name, cn=column_2, idf=id_column, my_id=some_id))
        id_exists = c.fetchone()
        if id_exists:
            print('5): {}'.format(id_exists))
        else:
            print('5): {} does not exist'.format(some_id))


In [42]:
retrievals(mydb, 'CONFERENCES', 'confName', 'confID', 'confID', 3 )

('1):', [(1, u'ECSA')])
('2):', [(u'ECSA',)])
('3):', [(u'ECSA', 1)])
('4):', [(1, u'ECSA')])
5): (3, u'WICSA')


In [ ]:
f = paperPK(mydb, 'PAPER', 'paperID')

In [ ]:
print keylist
